In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
import sys
sys.path.append(r"Q:\Model Projects\Freeway Network Managed Lanes\notebooks\survey")

In [ ]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import fiona
from fiona import open as fiona_open
from collections import Counter
import folium
from folium.plugins import PolyLineTextPath
import math
import csv
from datetime import datetime
from processed_survey import ProcessedSurvey
from shapely.geometry import Point, LineString, MultiLineString
from shapely.geometry import shape
from shapely.ops import nearest_points
from shapely.ops import linemerge, unary_union
from shapely.strtree import STRtree

import matplotlib.pyplot as plt

In [ ]:
## pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
## pd.reset_option('display.max_rows')
## pd.reset_option('display.max_columns')

In [ ]:
# Graphics
from matplotlib import colors
import seaborn as sns

# Add basemap (for mapping spatial data)
import contextily as ctx

sns.set_context(context='paper')

# Set random seed for reproducibility
from numpy.random import seed

import rioxarray  # raster processing
import xarray  # Surface data manipulation

## warnings is a module that allows you to filter warnings
import warnings
## we are going to ignore all warnings (so they won't print)
warnings.filterwarnings("ignore")

## MTC version

### function: plot the map (MTC version)

In [ ]:
def plot_matched_path_qc_map(trip_id_to_plot, df, df_match, path):
    """
    Plot matched path QC map for a single trip:
      - df:      full GeoDataFrame of matched_path_gdf (loaded once)
      - df_match:full GeoDataFrame of matched_gdf (loaded once)
      - path:full GeoDataFrame of path (loaded once)
    Returns a Folium Map object.
    """
    # Fast-load only the features for this trip using Pyogrio + Arrow and a WHERE filter
    gdf_path = (
        df[df["trip_id"] == trip_id_to_plot]
        .loc[:, ["trip_id", "rownum", "osmid", "name", "travel_time", "geometry", "origin_junction_id", "destination_junction_id"]]
        .set_crs("EPSG:3857")  # assign source CRS if not already set
        .to_crs(epsg=4326)     # reproject for mapping
    )
    
    # Fast‐load only the features for this trip from the matched_gdf layer
    gdf_match = (
        df_match[df_match["trip_id"] == trip_id_to_plot]
        .loc[:, ["trip_id", "rownum", "osmid", "name", "travel_time", "geometry"]]
        .set_crs("EPSG:3857")  # assign source CRS if not already set
        .to_crs(epsg=4326)     # reproject for mapping
    )

    #Preare for GPS subset
    gps_points = path[path["trip_id"] == trip_id_to_plot].copy()
    gps_points["collect_time"] = pd.to_datetime(gps_points["collect_time"])
    gps_points = gps_points.sort_values("collect_time").reset_index(drop=True)

    
    
    # -- Create map --
    if not gdf_path.empty:
        center = gdf_path.geometry.unary_union.centroid.coords[0]
        m = folium.Map(location=[center[1], center[0]], zoom_start=13, tiles="cartodbpositron")
    else:
        # center on mean GPS location
        lat0, lon0 = gps_points["lat"].mean(), gps_points["lon"].mean()
        m = folium.Map(location=[lat0, lon0], zoom_start=13, tiles="cartodbpositron")
    

    def draw_gps_points(m, gps_points):
       
        # 1) Find earliest and latest times
        first_time = gps_points["collect_time"].min()
        last_time  = gps_points["collect_time"].max()
        
        # 2) Loop and style
        for _, pt in gps_points.iterrows():
            info = (
                f"Trip ID: {pt['trip_id']}<br>"
                f"Time: {pt['collect_time']}<br>"
                f"Accuracy: {pt['accuracy']} m<br>"
                f"Bearing: {pt['bearing']}°<br>"
                f"Speed: {pt['speed']} m/s"
            )
            popup = folium.Popup(info, max_width=250, sticky=True)
            popup.options.update({"autoClose": False, "closeOnClick": True})
        
            # style by timestamp
            if pt["collect_time"] == first_time:
                color, radius = "green", 6
            elif pt["collect_time"] == last_time:
                color, radius = "red", 6
            else:
                color, radius = "black", 2
        
            folium.CircleMarker(
                location=[pt["lat"], pt["lon"]],
                radius=radius,
                color=color,
                fill=True,
                fill_opacity=0.8,
                popup=popup
            ).add_to(m)


    # -- Normalize and count repeated OSMIDs in path layer
    
    def normalize_osmid(x):
        try:
            if isinstance(x, list):
                return tuple(sorted(x))
            elif isinstance(x, str) and x.startswith('['):
                return tuple(sorted(eval(x)))
            else:
                return x
        except:
            return x

    
    # if gdf_path is NA, quit earlier
    if gdf_path.empty:
        draw_gps_points(m, gps_points)
        return m  # draw only GPS point

    
    gdf_path['routeID'] = gdf_path.apply(
    lambda x: '{}_{}'.format(x['origin_junction_id'], x['destination_junction_id']), axis=1)
    routeID_counts = Counter(gdf_path['routeID'])
    gdf_path['count'] = gdf_path['routeID'].map(routeID_counts)
    
    
    # -- Draw path lines with zoom‐responsive ▲ and =, with “=” halfway between ▶ --
    gdf_path_sorted = gdf_path.sort_values(by="rownum")
    for _, row in gdf_path_sorted.iterrows():
        count = row["count"] if "count" in row else 1
        weight = 3 + 2 * (count - 1)
        color = "#007AFF" if count == 1 else "#32fbe0"
        coords = [(pt[1], pt[0]) for pt in row.geometry.coords]
    
        # 1) Draw the base polyline
        # 1a) Add a popup to each segment showing its sequence (rownum) and OSMID
        popup = folium.Popup(
            f"Sequence: {row['rownum']}<br>OSMID: {row['osmid']}<br>routeID: {row['routeID']}",
            max_width=200, sticky = True
        )
        popup.options.update({
            "autoClose": False,
            "closeOnClick": True
        })
        
        poly = folium.PolyLine(
            locations=coords,
            weight=weight,
            color=color,
            opacity=0.8,
            popup = popup
        ).add_to(m)

    
        # 2) Add ▶ arrowheads (and "=") along the line, letting Leaflet.TextPath auto-rotate ▶ 
        arrow_layer = PolyLineTextPath(
            poly,
            text="=▶",
            repeat="50%",      # percent‐based spacing (initial)
            offset=15,          # pixels above the centerline
            orientation="auto",# auto-rotate the ▶ glyph along the segment
            attributes={
                "fill": "#969696",
                "font-size": "12px",
                "font-weight": "bold",
            }
        ).add_to(m)
    
    
    
        # 3) Inject JS to update repeat‐spacing on every zoom change
        arrow_name = arrow_layer.get_name()
        map_name = m.get_name()
    
        js = f"""
        <script>
        (function() {{
          var map   = {map_name};
          var arrow = {arrow_name};
    
          function updateRepeats() {{
            var z = map.getZoom();
            var newRepeat;
            if (z <= 10) {{
              newRepeat = '80%';
            }} else if (z === 11) {{
              newRepeat = '70%';
            }} else if (z === 12) {{
              newRepeat = '60%';
            }} else if (z === 13) {{
              newRepeat = '50%';
            }} else {{
              newRepeat = '40%';
            }}
    
            // Compute half‐offset (in percent) so “=” sits halfway between ▶ markers
            var repVal = parseInt(newRepeat, 10);
            var halfOffset = (repVal / 2) + '%';
    
            // Update the combined “=▶” text:
            //   - repeat at newRepeat% intervals
            //   - offset: 7 px above line
            arrow.setText(
              '=▶',
              {{ repeat: newRepeat, offset: 12, startOffset: halfOffset }}
            );
          }}
    
          // Run once immediately, then on each zoomend
          updateRepeats();
          map.on('zoomend', updateRepeats);
        }})();
        </script>
        """
        folium.Element(js).add_to(m)
    # ----------------------------------------------------------------------------
    
    
    # --draw vertical line to link GPS point and its nearest neighbor path--
    # 1) Merge all matched segments
    if not gdf_match.empty:
        matched_union = gdf_match.geometry.unary_union
    
        # 2) For each GPS fix, draw a little perpendicular connector
        for _, prow in gps_points.iterrows():
            gps_pt      = Point(prow["lon"], prow["lat"])
            dist_along  = matched_union.project(gps_pt)
            on_route_pt = matched_union.interpolate(dist_along)
            connector   = LineString([gps_pt, on_route_pt])
        
            # <-- Fix is here: unpack the coords tuple directly -->
            folium.PolyLine(
                locations=[(lat, lon) for lon, lat in connector.coords],
                color="black",
                weight=5,
                opacity=0.4,
            ).add_to(m)
        # ----------------------------------------------------------------------------

    
    # -- Overlay GPS points for that trip, highlighting by collect_time --

    draw_gps_points(m, gps_points)

    
    # -- Display map --
    return m

### Load the data

In [ ]:
PATHDIR = Path(r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2022\BATS 2023 TDS Conflation\OSM_match_v2')
GPKG = PATHDIR / 'tds_conflation_results.gpkg'
STUDY_LOCATIONS = r'Q:\Model Projects\Freeway Network Managed Lanes\data\ml_study_locations.csv'

In [ ]:
survey_args_2023 = {
    'dir':r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2022\Processed_20241127\reformat_2019_rmoveonly_v2\04-merge_skims\adj_weights',
    'hh':    {'file':'survey2023_hrecx_rewt_base2023.dat',
              'sep':' ',
              'weight':'hhexpfac'},
    'person':{'file':'survey2023_precx_rewt_base2023.dat',
              'sep':' ',
              'weight':'psexpfac'},
    'day':   {'file':'survey2023_pdayx_rewt_base2023.dat',
              'sep':' ',
              'weight':'pdexpfac'},
    'trip':  {'file':'survey2023_tripx_rewt_base2023.dat',
              'sep':' ',
              'weight':'trexpfac'},
    'tour':  {'file':'survey2023_tourx_rewt_base2023.dat',
              'sep':' ',
              'weight':'toexpfac'},
    'raw_dir': r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2022\Deliverable_20241127',
    'raw_hh':     {'file':'hh.csv',
                  'weight':'hh_weight'},
    'raw_person': {'file':'person.csv',
                  'weight':'person_weight'},
    'raw_day':    {'file':'day.csv',
                  'weight':'day_weight'},
    'raw_trip':   {'file':'trip.csv',
                  'weight':'trip_weight'},
}

In [ ]:
s = ProcessedSurvey(**survey_args_2023)
trip = s.trip.data.copy()
raw_trip = s._raw_trip.data.copy()
xref = raw_trip[['hhno','pno','trip_id','tsvid']]
trip = pd.merge(trip, xref, on=['hhno','pno','tsvid'], how='left')
path = pd.read_csv(r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2022\Deliverable_20241127\location.csv')

In [ ]:
df = gpd.read_file(GPKG, layer='matched_path_gdf', engine="pyogrio", use_arrow=True)

In [ ]:
df_match = gpd.read_file(GPKG, layer='matched_gdf', engine="pyogrio", use_arrow=True)

In [ ]:
### IF FIRST TIME USE, PLEASE RUN THE FOLLOWING 1) and 2) 

### extract all ids and save these into .csv
##1) Extract all unique trip_ids from your raw path table
#all_ids = path["trip_id"].drop_duplicates().reset_index(drop=True)

## 2) Persist to disk for reuse
#all_ids.to_frame(name="trip_id").to_csv(r".\iteration\all_trip_ids.csv", index=False)
#print(f"Saved {len(all_ids)} unique trip_ids → all_trip_ids.csv")

### Load all IDs
all_ids = pd.read_csv(r".\iteration\all_trip_ids.csv")["trip_id"]

### generating random samples

In [ ]:
### entry for entering a specific trip_id!!!!!!!!!!
#trip_id_to_plot = 2339500301020

In [ ]:
# 1) Sample 100
SAMPLE_SIZE, RANDOM_STATE = 100, 12345
sample_ids = all_ids.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE).reset_index(drop=True)

# 2) Read/initialize pointer
ptr_file = Path(r".\iteration\trip_pointer.txt")
if ptr_file.exists():
    pointer = int(ptr_file.read_text())
else:
    pointer = 0

# 3) Show history in‐notebook
print("✅ Reviewed so far:", sample_ids.iloc[:pointer].to_list())
print("⏭️ Remaining    :", sample_ids.iloc[pointer:].to_list())

# 4) Pick next ID
trip_id_to_plot = sample_ids.iloc[pointer]
print(f"\n🔎 Now plotting sample[{pointer}] = {trip_id_to_plot}")

# 5) Persist pointer
pointer = (pointer + 1) % SAMPLE_SIZE
ptr_file.write_text(str(pointer))

# 6) Optionally append to a log
log_file = Path(r".\iteration\review_log.csv")
if not log_file.exists():
    with log_file.open("w", newline="") as f:
        csv.writer(f).writerow(["timestamp","pointer","trip_id"])
with log_file.open("a", newline="") as f:
    csv.writer(f).writerow([datetime.now().isoformat(), pointer, trip_id_to_plot])

# 7) …then run your mapping code using trip_id_to_plot…

In [ ]:
m = plot_matched_path_qc_map(trip_id_to_plot, df, df_match, path)
m

## SFCTA version

### Load Data

In [ ]:
#load the raw file (both for MTC and SFCTA)
path = pd.read_csv(r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2022\Deliverable_20241127\location.csv')

In [ ]:
#Load the SFCTA trip data
df_sfcta = pd.read_csv(r"Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2022\Processed_20241127\reformat_2019_rmoveonly_v2\map_matching-sfcta\output\paths.csv")
df_sfcta

In [ ]:
osm2017 = gpd.read_file(r"Q:\GIS\Transportation\Roads\Bay_Area\OSM\OSM_20250624\ways.shp", engine="pyogrio", use_arrow=True)
osm2017

In [ ]:
df_merge = df_sfcta.merge(osm2017, left_on = "edge", right_on = "GID")
df_merge

In [ ]:
gdf_sfcta = gpd.GeoDataFrame(df_merge, geometry="geometry", crs=osm2017.crs)
gdf_sfcta

In [ ]:
# flip the geometry for rows with "reversed==1"
def reverse_geom(geom):
    # processing single line
    if isinstance(geom, LineString):
        return LineString(geom.coords[::-1])
    # processing multiline
    if isinstance(geom, MultiLineString):
        return MultiLineString([
            LineString(line.coords[::-1]) 
            for line in geom.geoms
        ])
    # keep other unchanged
    return geom

mask = gdf_sfcta['reversed'] == 1
gdf_sfcta.loc[mask, 'geometry'] = (
    gdf_sfcta.loc[mask, 'geometry']
           .apply(reverse_geom)
)

In [ ]:
gdf_sfcta_match = gdf_sfcta[gdf_sfcta["m_lat"].isna() == True]

In [ ]:
### IF FIRST TIME USE, PLEASE RUN THE FOLLOWING 1) and 2) 

### extract all ids and save these into .csv
##1) Extract all unique trip_ids from your raw path table
#all_ids_sfcta = path["trip_id"].drop_duplicates().reset_index(drop=True)

## 2) Persist to disk for reuse
#all_ids_sfcta.to_frame(name="trip_id").to_csv(r".\sfcta\iteration_sfcta\all_trip_ids_sfcta.csv", index=False)
#print(f"Saved {len(all_ids_sfcta)} unique trip_ids → all_trip_ids_sfcta.csv")

### Load all IDs
all_ids_sfcta = pd.read_csv(r".\sfcta\iteration_sfcta\all_trip_ids_sfcta.csv")["trip_id"]

### Function: Plot the map (SFCTA version)

In [ ]:
def plot_matched_path_qc_map_sfcta(trip_id_to_plot_sfcta, gdf_sfcta, gdf_sfcta_match, path):
    """
    Plot matched path QC map for a single trip:
      - gdf_sfcta:       full GeoDataFrame of sfcta 2023 data (loaded once)
      - gdf_sfcta_match: full GeoDataFrame of matched_gdf (loaded once)
      - path:            trip with GPS location (loaded once)
    Returns a Folium Map object.
    """
    # Fast-load only the features for this trip using a WHERE filter
    gdf_path = (
        gdf_sfcta[gdf_sfcta["trip_id"] == trip_id_to_plot_sfcta]
        .loc[:, ["reversed", "GID", "trip_id", "collect_time", "NAME", "OSM_ID", "point_id", "source", "target", "X1", "Y1", "X2", "Y2", "geometry"]]
        .set_crs(epsg=4326)     # reproject for mapping
    )
    gdf_path["collect_time"] = pd.to_datetime(gdf_path["collect_time"])
    
    # Fast‐load only the features for this trip from the matched_gdf layer
    gdf_match = (
        gdf_sfcta_match[gdf_sfcta_match["trip_id"] == trip_id_to_plot_sfcta]
        .loc[:, ["reversed", "GID", "trip_id", "collect_time", "NAME", "OSM_ID", "point_id", "source", "target", "X1", "Y1", "X2", "Y2", "geometry"]]
        .set_crs(epsg=4326)     # reproject for mapping
    )

    #Preare for GPS subset
    gps_points = path[path["trip_id"] == trip_id_to_plot_sfcta].copy()
    gps_points["collect_time"] = pd.to_datetime(gps_points["collect_time"])
    gps_points = gps_points.sort_values("collect_time").reset_index(drop=True)
    
    
    # -- Create map --
    if not gdf_path.empty:
        center = gdf_path.geometry.unary_union.centroid.coords[0]
        m = folium.Map(location=[center[1], center[0]], zoom_start=13, tiles="cartodbpositron")
    else:
        # center on mean GPS location
        lat0, lon0 = gps_points["lat"].mean(), gps_points["lon"].mean()
        m = folium.Map(location=[lat0, lon0], zoom_start=13, tiles="cartodbpositron")
    

    def draw_gps_points(m, gps_points):
       
        # 1) Find earliest and latest times
        first_time = gps_points["collect_time"].min()
        last_time  = gps_points["collect_time"].max()
        
        # 2) Loop and style
        for _, pt in gps_points.iterrows():
            info = (
                f"Trip ID: {pt['trip_id']}<br>"
                f"Time: {pt['collect_time']}<br>"
                f"Accuracy: {pt['accuracy']} m<br>"
                f"Bearing: {pt['bearing']}°<br>"
                f"Speed: {pt['speed']} m/s"
            )
            popup = folium.Popup(info, max_width=250, sticky=True)
            popup.options.update({"autoClose": False, "closeOnClick": True})
        
            # style by timestamp
            if pt["collect_time"] == first_time:
                color, radius = "green", 6
            elif pt["collect_time"] == last_time:
                color, radius = "red", 6
            else:
                color, radius = "black", 2
        
            folium.CircleMarker(
                location=[pt["lat"], pt["lon"]],
                radius=radius,
                color=color,
                fill=True,
                fill_opacity=0.8,
                popup=popup
            ).add_to(m)


    # -- Normalize and count repeated OSMIDs in path layer
    
    def normalize_osmid(x):
        try:
            if isinstance(x, list):
                return tuple(sorted(x))
            elif isinstance(x, str) and x.startswith('['):
                return tuple(sorted(eval(x)))
            else:
                return x
        except:
            return x

    
    # if gdf_path is NA, quit earlier
    if gdf_path.empty:
        draw_gps_points(m, gps_points)
        return m  # draw only GPS point

    
    gdf_path['routeID'] = gdf_path.apply(
    lambda x: '{}_{}'.format(x['source'], x['target']), axis=1)
    routeID_counts = Counter(gdf_path['routeID'])
    gdf_path['count'] = gdf_path['routeID'].map(routeID_counts)
    
    
    # -- Draw path lines with zoom‐responsive ▲ and =, with “=” halfway between ▶ --
    gdf_path_sorted =  gdf_path.sort_values("collect_time").reset_index(drop=True)
    for _, row in gdf_path_sorted.iterrows():
        count = row["count"] if "count" in row else 1
        weight = 3 + 2 * (count - 1)
        color = "#007AFF" if count == 1 else "#32fbe0"
        coords = [(pt[1], pt[0]) for pt in row.geometry.coords]
    
        # 1) Draw the base polyline
        # 1a) Add a popup to each segment showing its sequence and OSMID
        popup = folium.Popup(
            f"Time: {row['collect_time']}<br>Trip_ID: {row['trip_id']}<br>OSMID: {row['OSM_ID']}<br>Reversed: {row['reversed']}<br>GID: {row['GID']}<br>X1: {row['X1']}<br>Y1: {row['Y1']}<br>X2: {row['X2']}<br>Y2: {row['Y2']}<br>routeID: {row['routeID']}",
            max_width=200, sticky = True
        )
        popup.options.update({
            "autoClose": False,
            "closeOnClick": True
        })
        
        poly = folium.PolyLine(
            locations=coords,
            weight=weight,
            color=color,
            opacity=0.8,
            popup = popup
        ).add_to(m)

    
        # 2) Add ▶ arrowheads (and "=") along the line, letting Leaflet.TextPath auto-rotate ▶ 
        arrow_layer = PolyLineTextPath(
            poly,
            text="=▶",
            repeat="50%",      # percent‐based spacing (initial)
            offset=15,          # pixels above the centerline
            orientation="auto",# auto-rotate the ▶ glyph along the segment
            attributes={
                "fill": "#969696",
                "font-size": "12px",
                "font-weight": "bold",
            }
        ).add_to(m)
    
    
    
        # 3) Inject JS to update repeat‐spacing on every zoom change
        arrow_name = arrow_layer.get_name()
        map_name = m.get_name()
    
        js = f"""
        <script>
        (function() {{
          var map   = {map_name};
          var arrow = {arrow_name};
    
          function updateRepeats() {{
            var z = map.getZoom();
            var newRepeat;
            if (z <= 10) {{
              newRepeat = '80%';
            }} else if (z === 11) {{
              newRepeat = '70%';
            }} else if (z === 12) {{
              newRepeat = '60%';
            }} else if (z === 13) {{
              newRepeat = '50%';
            }} else {{
              newRepeat = '40%';
            }}
    
            // Compute half‐offset (in percent) so “=” sits halfway between ▶ markers
            var repVal = parseInt(newRepeat, 10);
            var halfOffset = (repVal / 2) + '%';
    
            // Update the combined “=▶” text:
            //   - repeat at newRepeat% intervals
            //   - offset: 7 px above line
            arrow.setText(
              '=▶',
              {{ repeat: newRepeat, offset: 12, startOffset: halfOffset }}
            );
          }}
    
          // Run once immediately, then on each zoomend
          updateRepeats();
          map.on('zoomend', updateRepeats);
        }})();
        </script>
        """
        folium.Element(js).add_to(m)
    # ----------------------------------------------------------------------------
    
    
    # --draw vertical line to link GPS point and its nearest neighbor path--
    # 1) set condition
    if not gdf_match.empty:
    
        # 2) For each GPS fix, draw a little perpendicular connector
        for _, prow in gps_points.iterrows():
            gps_pt  = Point(prow["lon"], prow["lat"])
            
            # vectorize the distance to look for the nearest path
            dists = gdf_match.geometry.distance(gps_pt)
            idx_min = dists.idxmin()
            nearest_line = gdf_match.loc[idx_min, "geometry"]
            
            _, on_route_pt = nearest_points(gps_pt, nearest_line)
            
            folium.PolyLine(
                locations=[(lat, lon) for lon, lat in [gps_pt.coords[0], on_route_pt.coords[0]]],
                color="black", weight=5, opacity=0.4
            ).add_to(m)
        # ----------------------------------------------------------------------------

    
    # -- Overlay GPS points for that trip, highlighting by collect_time --

    draw_gps_points(m, gps_points)

    
    # -- Display map --
    return m

### generating random samples

In [ ]:
### entry for entering a specific trip_id!!!!!!!!!!
#trip_id_to_plot_sfcta = 2339500301020

In [ ]:
# 1) Sample 100
SAMPLE_SIZE, RANDOM_STATE = 100, 12345
sample_ids = all_ids_sfcta.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE).reset_index(drop=True)

# 2) Read/initialize pointer
ptr_file = Path(r".\sfcta\iteration_sfcta\trip_pointer.txt")
if ptr_file.exists():
    pointer = int(ptr_file.read_text())
else:
    pointer = 0

# 3) Show history in‐notebook
print("✅ Reviewed so far:", sample_ids.iloc[:pointer].to_list())
print("⏭️ Remaining    :", sample_ids.iloc[pointer:].to_list())

# 4) Pick next ID
trip_id_to_plot_sfcta = sample_ids.iloc[pointer]
print(f"\n🔎 Now plotting sample[{pointer}] = {trip_id_to_plot_sfcta}")

# 5) Persist pointer
pointer = (pointer + 1) % SAMPLE_SIZE
ptr_file.write_text(str(pointer))

# 6) Optionally append to a log
log_file = Path(r".\sfcta\iteration_sfcta\review_log.csv")
if not log_file.exists():
    with log_file.open("w", newline="") as f:
        csv.writer(f).writerow(["timestamp","pointer","trip_id"])
with log_file.open("a", newline="") as f:
    csv.writer(f).writerow([datetime.now().isoformat(), pointer, trip_id_to_plot_sfcta])

# 7) …then run your mapping code using trip_id_to_plot…
#---------------------------

In [ ]:
n = plot_matched_path_qc_map_sfcta(trip_id_to_plot_sfcta, gdf_sfcta, gdf_sfcta_match, path)
n